In [4]:
#Import the necessary modules
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [65]:
#Create the base url  
base_url = 'https://www.realtytrac.com/'

#Create a search for the LA area
la_search = 'mapsearch/sold/ca/los-angeles-county/'

In [66]:
#Grab the website content and store it in a BeautifulSoup object
response = requests.get(base_url+la_search)
soup = BeautifulSoup(response.content, 'html.parser')

In [71]:
houses = soup.find_all('div', class_="house alt clearfix")
#Number of pages to run the scraper through
num_pages = 20

print(f'The number of houses you will scrape is {len(houses)*num_pages}')

The number of houses you will scrape is 200


In [68]:
absolute_links = []

for i in range(num_pages):
    page = soup.find_all('div', class_="house alt clearfix")
    
    for house in page:
        absolute_links.append(base_url+house.find('a')['href'])

In [76]:
#Create the function that extracts the attributes for each house(link)

df = pd.DataFrame(columns=['Address', 'Property_type'])

def find_details(house):
    #Address
    address = house.find('span', class_='address heading').text
    
    #Property Details
    details = house.find('ul', class_='detail-list')
    add_details = details.find_all('li', itemprop='additionalProperty' )
    
    #Residential Bilding Types
    prop_type = details.find('span', itemprop='name').text
    
    #Rooms
    rooms = details.find('span', itemprop='description').text
    
    #Home Size
    home_size = add_details[0].find('span', itemprop='value').text
    home_unit = add_details[0].find('span', itemprop='unitText').text
    
    #Lot Size
    lot_size = add_details[1].find('span', itemprop='value').text
    lot_unit = add_details[1].find('span', itemprop='unitText').text
    
    #Year Built
    year_built = add_details[2].find('span', itemprop='value').text
    
    #Parcel Number
    parcel = add_details[3].find('span', itemprop='propertyID').text
    
    #RealtyTracPropteryID
    realty = add_details[4].find('span', itemprop='value').text
    
    #County
    county = add_details[5].find('span', itemprop='description').text
    
    #Subdivision
    sub = add_details[6].find('span', itemprop='value').text
    
    #Census
    census = add_details[7].find('span', itemprop='value').text
    
    #Tract
    tract = add_details[8].find('span', itemprop='value').text
    
    #Lot
    try:
        lot = add_details[9].find('span', itemprop='value').text
    except:
        lot = None
    
    #Zoning
    zoning = details.find_all('li')[12].text
    
    #Sales History
    sales = soup.find('section', id='occupancy-link').find_all('tr')

    date = sales[5].find('td', class_='col2').text #Date
    sale_price = sales[4].find('td', class_='col2').text #Sale Price
    
    #Estimated Value
    est = house.find('div', class_='rate-row access')
    try:
        estimate = est.find('strong', class_='price').text
    except:
        estimate= None
    
    #Home Disclosures
    disc = house.find('div', id="collapsePropertyHomeDisclosuresArea")
    
    #Need help rendering the javascript portion
    
    #sex_offenders = disc.find('li', class_='offenders alert').find('span', class_='hd-label').text
    #crime_index = disc.find('li', class_='crime concern').find('span', class_='hd-label').text
    #env_hazards = disc.find('li', class_='enviornmental alert').find('span', class_='hd-label').text
    #natural_disasters = disc.find('li', class_='disaster alert').find('span', class_='hd-label').text
    #school_index = disc.find('li', class_='schools alert').find('span', class_='hd-label').text
    
    #Create the df to be returned
    df = pd.DataFrame({
                       'Address':[address],
                       'Property_type': [prop_type],
                       'Rooms': [rooms],
                       'Home_size': [home_size+home_unit],
                       'Lot Size': [lot_size+lot_unit],
                       'Year_built': [year_built],
                       'Parcel_number': [parcel],
                       'RealtyID': [realty],
                       'County': [county],
                       'Subdivision': [sub],
                       'Census': [census],
                       'Tract': [tract],
                       'Lot': [lot],
                       'Zoning': [zoning],
                       'Date': [date],
                       'Sale_price': [sale_price],
                       'Estimated Value': [estimate]
                       #'Sex_Offenders': [sex_offenders],
                       #'Crime_index': [crime_index],
                       #'Enviornmental_Hazards': [env_hazards],
                       #'School Quality': [school_index]
                      })
    
    return df  

In [77]:
#Loop through the houses and grab all of the data

for i in range(len(absolute_links)):
    house_detail = requests.get(absolute_links[i])
    soup = BeautifulSoup(house_detail.content, 'html.parser')
    df = df.append(find_details(soup), ignore_index=True)
    if i%50 ==0:
        print(f'Scraped {df.shape[0]} houses')

Scraped 50 houses


In [86]:
#Save the dataframe to a csv file
df.to_csv('house_data.csv', index=False)